In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install nltk==3.5.0
!pip install wordnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 13.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434691 sha256=315ea351dd3a2f0441a173fa7429e5df45edcb8e63287fcab20e81c3801d39f2
  Stored in directory: /root/.cache/pip/wheels/13/ae/bb/5e2a232ebaa1d2f38dd5f587e9fc4cf6ccb12758d14dac14d8
Successfully

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import random

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

[nltk_data] Downloading package wordnet to /root/nltk_data...


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:
def generate_question(context, answer):
    input_text = f"context: {context} answer: {answer}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids,
                             max_length=128,
                             num_beams=4,
                             early_stopping=True)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

def generate_distractors(question, answer):
    
    question_tokens = question.split()
    answer_tokens = answer.split()

    
    synonyms = set()
    hypernyms = set()
    for token in answer_tokens:
        for synset in wn.synsets(token):
            for lemma in synset.lemmas():
                synonyms.add(lemma.name())
            for hypernym in synset.hypernyms():
                hypernyms.add(hypernym.lemma_names()[0])

    
    distractors = []
    while len(distractors) < 6:
        # Randomly choose between synonyms and hypernyms
        if random.random() < 0.5 and len(synonyms) > 0:
            distractor = random.choice(list(synonyms))
            synonyms.remove(distractor)
        elif len(hypernyms) > 0:
            distractor = random.choice(list(hypernyms))
            hypernyms.remove(distractor)
        else:
            break

        # Make sure distractor is not the same as t
        if distractor not in answer_tokens and distractor not in distractors:
            distractors.append(distractor)

    return distractors

In [ ]:
context = "Chibueze was a good boy before he turned bad because of his friends till now he is regretting it ."
answer = "good boy"
question = generate_question(context, answer)
distractors = generate_distractors(question, answer)
random_index = random.randint(0, len(distractors))  # generate a random index within the range of the list
distractors.insert(random_index, answer)
print(question)
print(distractors)

question: Was Chibueze a good or bad boy?
['secure', 'Black_man', 'good boy', 'male', 'morality', 'quality', 'male_offspring']
